In [ ]:
import pandas as pd
import geopandas as gp 
import json
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(rc={'figure.figsize':(12,8)})
%matplotlib inline
from tqdm import tqdm
import glob
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import linregress
import pickle as pkl
import networkx as nx 

In [ ]:
drug_association_graph = nx.read_gexf('../mappings/drug_association_graph.gexf')
drug_cat_association_graph = nx.read_gexf('../mappings/category_association_graph.gexf')

In [ ]:
# with open('../mappings/Bipartite_Drug_graph.pkl','rb') as f:
#     drug_association_graph  = pkl.load(f)

# with open('../mappings/Bipartite_Drug_category_graph.pkl','rb') as f:
#     drug_cat_association_graph  = pkl.load(f)

In [ ]:
# nx.write_gexf(drug_association_graph, "../mappings/drug_association_graph.gexf")

In [ ]:
# nx.write_gexf(drug_cat_association_graph, "../mappings/category_association_graph.gexf")

In [ ]:
def cleanStringofUTF(string):
    my_str_as_bytes = str.encode(string,'utf-8')
    cleaned = str(my_str_as_bytes).replace('\xe8','e').replace('\xf6','o')
    return cleaned

def enrichdrugs(chem_dict , drugs):
    diabetes_drug_words = [drugs[k]['name'].lower() for k in drugs]
    for drug in chem_dict:
        Name = chem_dict[drug]['name'].replace('(','').replace(')','')
        slot1 = Name.lower().split('/')
        slot2 = Name.lower().split(' ')
        slot3 = Name.lower().split(' & ')
        common1 = set(diabetes_drug_words).intersection(slot1)
        common2 = set(diabetes_drug_words).intersection(slot2)
        common3 = set(diabetes_drug_words).intersection(slot3)
        
        if len(common1) > 0 or len(common2) > 0 or len(common3) > 0:
#             print common1 , common2 , common3
            drugs[chem_dict[drug]['code']] = {'disease':'' , 'disease_given_drug':0.0 , 'matched_disease':'', 'name':chem_dict[drug]['name'].strip() }

            
            
def makeChemDict(BNF_Chem):
    chem_dict = {}
    for index, row in BNF_Chem.iterrows():
        chem_dict[row['UNII_drugbank']] = {}
        chem_dict[row['UNII_drugbank']]['name'] = row['NAME']
        chem_dict[row['UNII_drugbank']]['code'] = row['BNF_code']
    return chem_dict
    
def getDrugCategory(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Categories']) > 0:
            for cat in drugbankDict[k]['Categories']:
                matched_memo = []
                catString = cat.values()[0]#.split('\u2014')[-1]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:# == len(categorylist):
                        allMatched.append(k)
#                         print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return list(set(allMatched)) , drugs


def getDrugforDiseaseDrugbank(categorylist, BNF_Chem, drugbankDict):
    allMatched = []
    drugs = {}
    chem_dict = makeChemDict(BNF_Chem)
    
    for k in drugbankDict:
        if len(drugbankDict[k]['Associations']) > 0:
            for cat in drugbankDict[k]['Associations']:
                matched_memo = []
                catString = cat.values()[0]
                t = catString.lower().strip()
                for categoryString in categorylist:
                    categoryString = categoryString.lower()
                    if t.find(categoryString) >= 0:
                        matched_memo.append(categoryString)
                if k in chem_dict:
                    if len(matched_memo) > 0:
                        allMatched.append(k)
#                         print chem_dict[k]
                        drugs[chem_dict[k]['code']] = {}
                        drugs[chem_dict[k]['code']]['name'] = chem_dict[k]['name']
                        drugs[chem_dict[k]['code']]['matched_cat'] = categorylist
    enrichdrugs(chem_dict,drugs)               
    return  allMatched , drugs


def findDrugsForDisease(Graph, Disease, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Disease.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Disease.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'symptom':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_disease'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['disease'] = Disease
    enrichdrugs(chem_dict,drugs)
    return drugs


def generateConfidence(drugs,Graph):
    shared = []
    All = []
    denom = max(Graph.degree().values())
    for d in drugs:
        name = drugs[d]['name']
        for e in Graph.edges(data=True):
            if Graph.node[e[0]]['type'] == 'symptom':
                if e[1] == name:
                    shared.append(Graph.degree()[e[1]]-1)
                else:
                    continue
            else:
                
                if e[0] == name:
                    shared.append(Graph.degree()[e[0]]-1)
                else:
                    continue
#     shared = [float(k) for k in shared]
    num = [k for k in shared if k > 1]

    return float(len(num)+1.0)/float(len(shared)+1.0)
#     return float(len(num))/float(len(shared)) * 10.0
#     return len(num)
                
                
def findDrugsForCategory(Graph, Cat, BNF_Chem ):#,threshProb):
    chem_dict = makeChemDict(BNF_Chem)
    drugs = {}
    for e in Graph.edges(data=True):
        if (cleanStringofUTF(e[1]).lower().find(Cat.lower()) >=0) or (cleanStringofUTF(e[0]).lower().find(Cat.lower()) >= 0) :
            drugNode = ''
            matchedDisease = ''
            if Graph.node[e[0]]['type'] == 'category':
                drugNode = e[1]
                matchedDisease = e[0]
            else:
                drugNode = e[0]
                matchedDisease = e[1]
#             print Graph.node[drugNode]['Id']
            drugs[Graph.node[drugNode]['Id']] = {}
            drugs[Graph.node[drugNode]['Id']]['name'] = drugNode
            drugs[Graph.node[drugNode]['Id']]['matched_cat'] = matchedDisease
            drugs[Graph.node[drugNode]['Id']]['category'] = Cat
    enrichdrugs(chem_dict,drugs)
    return drugs


In [ ]:
files = glob.glob('../../BL_Work/openPrescribe/serialized/*.gz')
print(files)

In [ ]:
chem = pd.read_csv('../mappings/CHEM_MASTER_MAP.csv')
len(chem)

# chem = chem.dropna()

matched = chem[chem['UNII_drugbank']!='[]']

matchedMap = {}
for index,row in matched.iterrows():
    if row['UNII_drugbank'] not in matchedMap:
         matchedMap[row['UNII_drugbank']] = []
    matchedMap[row['UNII_drugbank']].append(row['BNF_code'])

diseases = [
 "anxiety",
 "rheumatoid",
 "osteoporosis",
 "depression",
 "diabetes",
 "stroke",
 "hypertension",
 "chronic obstructive pulmonary disease", 
 "dementia",
 "asthma",
 "sleeplessness",
]

DiseaseDrugs = {}
for d in diseases:
    drugs = findDrugsForDisease(drug_association_graph,d ,chem)
#     _ , drugs = getDrugforDiseaseDrugbank([d] ,chem,drugbank_dict)
    for drug in drugs:
        DiseaseDrugs[drug] = {}
        DiseaseDrugs[drug]['chemName'] = drugs[drug]['name']
        DiseaseDrugs[drug]['disease'] = d

In [ ]:
# findDrugsForDisease(drug_association_graph,'sleeplessness',chem)

In [ ]:
DiseaseDrugs

In [ ]:
disease_drug_map = {}
for k in DiseaseDrugs:
    if DiseaseDrugs[k]['disease'] not in disease_drug_map:
        disease_drug_map[DiseaseDrugs[k]['disease']] = []
    disease_drug_map[DiseaseDrugs[k]['disease']].append(k)

In [ ]:
drug_map_dict = {'BNF_code':[], 'Drug_name':[] , 'Mapped_Condition': []}
for k in DiseaseDrugs:
    drug_map_dict['BNF_code'].append(k)
    drug_map_dict['Drug_name'].append(DiseaseDrugs[k]['chemName'])
    drug_map_dict['Mapped_Condition'].append(DiseaseDrugs[k]['disease'])
drug_map_df = pd.DataFrame.from_dict(drug_map_dict)    
drug_map_df.to_csv('data_prep/Drugs.csv',index=False)

In [ ]:
disease_drug_map

In [ ]:
LSOA_dist = json.load(open('../mappings/GP_LSOA_PATIENTSDIST.json','rb'))

In [ ]:
LSOA_dist_2021 = json.load(open('mappings/GP_LSOA_PATIENTSDIST_2021.json','rb'))

In [ ]:
LSOA_dist['A81001']['E01033477']

In [ ]:
LSOA_dist_2021['A81001']['E01033477']

In [ ]:
# LSOA_dist_new = pd.read_csv('mappings/gp-reg-pat-prac-lsoa-all.csv')

In [ ]:
# LSOA_dist_new.head()

In [ ]:
# LSOA_dist_2021 = {}
# for name , group in LSOA_dist_new.groupby('PRACTICE_CODE'):
#     LSOA_dist_2021[name] = {}
#     total = sum(group['Number of Patients'])
#     for index , row in group.iterrows():
#         LSOA_dist_2021[name][row['LSOA_CODE']] = float(row['Number of Patients'])/float(total)
    
        

In [ ]:
# json.dump(LSOA_dist_2021 , open('mappings/GP_LSOA_PATIENTSDIST_2021.json','w'))

In [ ]:
%store -r taxonomyDict

In [ ]:
# json.dump(taxonomyDict, open('mappings/taxomomy_dict.json','w'))

In [ ]:
%store -r cityMap

In [ ]:
# json.dump(cityMap, open('mappings/City_map_dict.json','w'))

In [ ]:
ward_pop = pd.read_csv('../mappings/ward_pop.csv')

In [ ]:
population = {}
for index, row in ward_pop.iterrows():
    population[row['Ward Code 1']] = float(row['All Ages'].replace(',',''))

In [ ]:
df_city = pd.read_csv('../mappings/lower_layer_super_output_area_2011_to_major_towns_and_cities_december_2015_lookup_in_england_and_wales.csv')

In [ ]:
df_city.head()

In [ ]:
cityMap = {}
for name , group in df_city.groupby('TCITY15NM'):
        cityMap[name] = list(group['LSOA11CD'])

In [ ]:
LSOA_survey_takers = json.load(open('../mappings/LSOA_suvery_pop.json'))

In [ ]:
disease_drugs = json.load(open("../mappings/Disease_Drug_DrugBank.json",'rb'))

In [ ]:
drugbank_dict = json.load(open('../mappings/Drugbank_drugs_data.json','rb'))

In [ ]:
cityMap

In [ ]:
IMD_df = pd.read_csv('../../BL_Work/File_7_ID_2015_All_ranks__deciles_and_scores_for_the_Indices_of_Deprivation__and_population_denominators.csv')

In [ ]:
IMD_df.head()

In [ ]:
LSOA_pop = {}
LSOA_IMD = {}
for index, row in IMD_df.iterrows():
    LSOA_pop[row['LSOA code (2011)']] = row['Total population: mid 2012 (excluding prisoners)']
    LSOA_IMD[row['LSOA code (2011)']] = row['Index of Multiple Deprivation (IMD) Score']

In [ ]:
len(LSOA_pop.keys())

In [ ]:
cityPop = {}
city_IMD = {}
city_survey_pop = {}
for k in cityMap:
    pop = 0
    surveypop = 0
    IMD = []
    for j in cityMap[k]:
        try:
            pop += LSOA_pop[j]
            surveypop += LSOA_survey_takers[j]
            IMD.append(LSOA_IMD[j])
        except:
            print("could not find LSOA",j)
    city_IMD[k] = {}
    if pop > 0:
        cityPop[k] = pop
        city_survey_pop[k] = surveypop
        city_IMD[k]['median_IMD'] = np.median(IMD)
        city_IMD[k]['mean_IMD'] = np.mean(IMD)

In [ ]:
LSOA_patient_pop = {}
LSOA_patients_map = json.load(open('data_prep/GPs.json','r'))
for GP in tqdm(LSOA_patients_map):
    for lsoa in LSOA_patients_map[GP]['Patient_registry_LSOA']:
        if lsoa not in LSOA_patient_pop:
            LSOA_patient_pop[lsoa] = LSOA_patients_map[GP]['Patient_registry_LSOA'][lsoa]
        else:
            LSOA_patient_pop[lsoa] += LSOA_patients_map[GP]['Patient_registry_LSOA'][lsoa]

In [ ]:
sum(LSOA_patient_pop.values())

In [ ]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='dataPrep_postcovid.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

In [ ]:
def calculateTemporalMetrics_LSOA(all_presc , old = True):
    LSOA_dosage = {}
    LSOA_costs = {}
    LSOA_items = {}
    LSOA_quantity = {}

    LSOA_patient_count = {}
    fail = 0.0
    LSOA_map = {}
    [LSOA_dist_old , LSOA_dist_2021] = loadLSOA_mappings()

    #At this time we are using the same map for all files. Ideally, every year needs to have a different map.
    if old:
        quantityField = '8'
        dosageField = '19'
        costField = '7'
        practiceField = '2'
        itemField = '5'
        LSOA_map = LSOA_dist_2021
    else:
        quantityField = 'TOTAL_QUANTITY'
        dosageField = '19'
        costField = 'ACTUAL_COST'
        practiceField = 'PRACTICE_CODE'
        itemField = 'ITEMS'
        LSOA_map = LSOA_dist_2021

    for name, group in all_presc.groupby(practiceField):
        total_dosage = np.sum(group[dosageField])
        total_cost = np.sum(group[costField])
        total_quantity = np.sum(group[quantityField])
        total_items = np.sum(group[itemField])

        if name in LSOA_map:        
            for k in LSOA_map[name]:
                if k not in LSOA_dosage:
                    LSOA_dosage[k] = 0.0
                    LSOA_costs[k] = 0.0
                    LSOA_quantity[k] = 0.0
                    LSOA_items[k] = 0.0
                LSOA_dosage[k]+= float(total_dosage)*float(LSOA_map[name][k])
                LSOA_quantity[k]+= float(total_quantity)*float(LSOA_map[name][k])
                LSOA_items[k]+= float(total_items)*float(LSOA_map[name][k])
                LSOA_costs[k]+= float(total_cost)*float(LSOA_map[name][k])
    
    return  LSOA_quantity , LSOA_costs, LSOA_dosage , LSOA_items

In [ ]:
pdp1 = pd.read_csv('../../BL_Work/openPrescribe/serialized/201910.gz',compression='gzip')

In [ ]:
pdp2 = pd.read_csv('../../BL_Work/openPrescribe/serialized/201710.gz',compression='gzip')

In [ ]:
pdp1[['BNF_CODE', 'BNF_DESCRIPTION',
       'BNF_CHAPTER_PLUS_CODE', 'QUANTITY', 'ITEMS', 'TOTAL_QUANTITY',
       'ADQUSAGE', 'NIC', 'ACTUAL_COST']]

In [ ]:
pdp2.columns
pdp2[['1', '2', '3', '4', '5', '6', '7', '8', '9']]

In [ ]:
del pdp , pdp1 , pdp2

In [ ]:
# monthly_borough_dosage = {}
# monthly_borough_costs = {}

monthly_borough_dosage_new = {}
monthly_borough_costs_new = {}
monthly_borough_quantity_new = {}
monthly_borough_items_new = {}

In [ ]:
files.sort()

In [ ]:
files[53:]

In [ ]:
# files[107:]

In [ ]:
for f in tqdm(files[53:]):
    month = f.split('/')[-1].split('.')[0]
    logging.debug("Working with month  " + month)
    if int(month) > 201906:
        old = False
    else:
        old = True
    
    monthly_borough_dosage_new[month] = {}
    monthly_borough_costs_new[month] = {}
    monthly_borough_quantity_new[month] = {}
    monthly_borough_items_new[month] = {}
    pdp = pd.read_csv(f,compression='gzip')
    for disease in tqdm(disease_drug_map):
        print("Working with disease  " + disease)
        logging.debug("Working with disease  " + disease)
        monthly_borough_dosage_new[month][disease] = {}
        monthly_borough_costs_new[month][disease] = {}
        monthly_borough_quantity_new[month][disease] = {}
        monthly_borough_items_new[month][disease] = {}
        drugs = disease_drug_map[disease]
        opioids = pdp.loc[pdp['16'].isin(drugs)] #Original opioids

        monthly_borough_quantity_new[month][disease] , monthly_borough_costs_new[month][disease], monthly_borough_dosage_new[month][disease]  , monthly_borough_items_new[month][disease] = calculateTemporalMetrics_LSOA(opioids, old)
   

In [ ]:
pdp.head(n=5)

In [ ]:
monthly_borough_dosage_new['202002']['anxiety'].keys()

In [ ]:
monthly_borough_dosage_new['202002']['anxiety']['E01015028']

In [ ]:
monthly_borough_dosage_total= monthly_borough_dosage_new.copy()
# for yyyymm in tqdm(monthly_borough_dosage_total):
#     for d in monthly_borough_dosage_total[yyyymm]:
#         for lsoa in  monthly_borough_dosage_total[yyyymm][d]:
#             monthly_borough_dosage_total[yyyymm][d][lsoa] = (monthly_borough_dosage_total[yyyymm][d][lsoa]/1000.0)*LSOA_patient_pop[lsoa]

In [ ]:
for disease in tqdm(disease_drug_map.keys()):
    disease_dict = {'YYYYMM':[] , 'LSOA_CODE' : [] , 'Total_prescriptions' : [] ,'Dosage_ratio' :[] , 'Patient_count' : []}
    for yyyymm in monthly_borough_dosage_total:
        for LSOA_CODE in monthly_borough_dosage_total[yyyymm][disease]:
            if LSOA_CODE[0] == 'E':
                disease_dict['YYYYMM'].append(yyyymm)
                disease_dict['LSOA_CODE'].append(LSOA_CODE)
                disease_dict['Total_prescriptions'].append(monthly_borough_dosage_total[yyyymm][disease][LSOA_CODE])
                disease_dict['Dosage_ratio'].append(monthly_borough_costs_new[yyyymm][disease][LSOA_CODE])
                disease_dict['Patient_count'].append(LSOA_patient_pop[LSOA_CODE])
    disease_df = pd.DataFrame.from_dict(disease_dict)
    filename = 'data_prep/'+disease+'_V2.csv.gz'
    disease_df.to_csv(filename,index=False,compression='gzip')
            

In [ ]:
disease_drug_map

In [ ]:

for bnf in disease_drug_map['anxiety']:
    print(DiseaseDrugs[bnf])

In [ ]:
# json.dump(monthly_borough_dosage_new,open('../mappings/pre_post_monthy_presc_pre072019.json','w'))

In [ ]:
# json.dump(monthly_borough_costs_new,open('../mappings/pre_post_monthy_cost_pre072019.json','w'))

In [ ]:
# json.dump(monthly_borough_dosage_new,open('../mappings/pre_post_monthy_presc_post072019.json','w'))

In [ ]:
# json.dump(monthly_borough_costs_new,open('../mappings/pre_post_monthy_cost_post072019.json','w'))

In [ ]:
monthly_dosage_BL = json.load(open('../mappings/pre_post_monthy_presc_pre072019.json','r'))
monthly_dosage_COVID = json.load(open('../mappings/pre_post_monthy_presc_post072019.json','r'))

In [ ]:
monthly_dosage_COVID['202001'].keys()

In [ ]:
BL_months = [
             #['201101', '201102', '201103', '201104', '201105', '201106', '201107', '201108', '201109', '201110', '201111', '201112'],
             #['201201', '201202', '201203', '201204', '201205', '201206', '201207', '201208', '201209', '201210', '201211', '201212'],
             #['201301', '201302', '201303', '201304', '201305', '201306', '201307', '201308', '201309', '201310', '201311', '201312'],
             #['201401', '201402', '201403', '201404', '201405', '201406', '201407', '201408', '201409', '201410', '201411', '201412'],
             ['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512'],
             ['201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612'],
             ['201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712'],
             ['201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812']]
#             

COVID_months = ['202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012']

In [ ]:
print sorted(monthly_dosage_COVID['202007']['diabetes'].keys())

In [ ]:
from collections import OrderedDict

d_l = ['anxiety',
 'heart failure',
 'rheumatoid',
 'epilepsy',
 'dementia',
 'stroke',
 'hypertension',
 'diabetes',
 'chronic obstructive pulmonary disease',
#  'obesity',
 'coronary artery disease',
#  'kidney disease',
 'depression',
 'osteoporosis']

# d_l = ['anxiety',
#  'depression']
month_names = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
# selected_disease = 'stroke'
city = 'London'

diseaseStats = {}
for d in d_l:
    selected_disease = d
    monthly_BL_numbers = OrderedDict()
    for month in range(len(BL_months[0])):
        monthly_BL_numbers[month_names[month]] = []
        for year in range(len(BL_months)):
            if city in monthly_dosage_BL[BL_months[year][month]][selected_disease]:
                monthly_BL_numbers[month_names[month]].append(monthly_dosage_BL[BL_months[year][month]][selected_disease][city])
            else:
                print("no prescriptions for disease " + d)
                monthly_BL_numbers[month_names[month]].append(0.0)
    
    diseaseStats[d] = monthly_BL_numbers  
        
    


In [ ]:
# monthly_dosage_COVID[COVID_months[0]]['depression'][city]

In [ ]:
diseaseStats.keys()

In [ ]:
# diseaseStats['depression']

In [ ]:
diseaseZ_scores = {}
for d in d_l:
    Z_scores = []
    for i in range(len(COVID_months)):
        mean = np.mean(diseaseStats[d][month_names[i]])
        sigma = np.std(diseaseStats[d][month_names[i]])
        if sigma > 0:
            if city in monthly_dosage_COVID[COVID_months[i]][d]:
                Z_scores.append(10 + (monthly_dosage_COVID[COVID_months[i]][d][city]-mean)/sigma)
            else:
                print("no prescriptions for disease " + d)
                Z_scores.append(5 + (-mean)/sigma)
    diseaseZ_scores[d] = Z_scores

In [ ]:
# diseaseZ_scores

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 12)
sns.set_style("white")
legend = []
for d in diseaseZ_scores:
    sns.lineplot(y=diseaseZ_scores[d],x=np.arange(0,len(diseaseZ_scores[d]),1))
    legend.append('Z for ' + d)
ax = plt.gca()

plt.xticks(np.arange(0,12,1),month_names,rotation=90)

plt.legend(legend, fontsize = 10)
plt.xlabel("Months of the year 2020",fontsize=20)
plt.ylabel("Z scores for prescriptions per 1000",fontsize=20)
plt.title("Prescription Z scores for city of " + city, fontsize=20)

In [ ]:
monthly_dosage_COVID

In [ ]:
city = 'London'

d_l = ['anxiety',
 'heart failure',
 'rheumatoid',
 'epilepsy',
 'dementia',
 'stroke',
 'hypertension',
 'diabetes',
 'chronic obstructive pulmonary disease',
#  'obesity',
 'coronary artery disease',
#  'kidney disease',
 'depression',
 'osteoporosis']

# d_l = ['anxiety' , 'depression']

all_covid_months = ['201907','201908','201909','201910','201911','201912','202001', '202002', '202003', '202004', '202005', 
                    '202006', '202007', '202008', '202009', '202010', '202011', '202012','202101','202102',]

disease_timelines = {}
months = monthly_dosage_COVID.keys
for disease in d_l:
    disease_timelines[disease] = []
    for month in monthly_dosage_COVID:
        if city in monthly_dosage_COVID[month][disease]:
            disease_timelines[disease].append(    monthly_dosage_COVID[month][disease][city] )

In [ ]:
# disease_timelines

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 12)
sns.set_style("white")
legend = []
for d in disease_timelines:
    sns.lineplot(y=disease_timelines[d],x=np.arange(0,len(disease_timelines[d]),1))
    legend.append('Prescriptions for ' + d)
ax = plt.gca()

plt.xticks(np.arange(0,20,1),all_covid_months,rotation=90)

plt.legend(legend, fontsize = 10)
plt.xlabel("Months of the year 2020",fontsize=20)
plt.ylabel("Z scores for prescriptions per 1000",fontsize=20)
plt.title("Prescriptions for city of " + city, fontsize=20)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(16, 12)
sns.set_style("white")
legend = []
for d in disease_timelines:
    if len(disease_timelines[d])> 0:
        mean = (np.mean(disease_timelines[d]))
        sigma = (np.std(disease_timelines[d]))
        Z_scores = [float(k - mean)/sigma for k in disease_timelines[d]]
        sns.lineplot(y=Z_scores,x=np.arange(0,len(disease_timelines[d]),1))
        legend.append('Vanilla Z score for ' + d)
ax = plt.gca()

plt.xticks(np.arange(0,20,1),all_covid_months,rotation=90)

plt.legend(legend, fontsize = 10)
plt.xlabel("Months of the year 2020",fontsize=20)
plt.ylabel("vanilla Z scores for prescriptions per 1000",fontsize=20)
plt.title("Prescription Z scores for city of " + city, fontsize=20)